# Problem description

Five times more deadly than the flu, COVID-19 causes significant morbidity and mortality. Like other pneumonias, pulmonary infection with COVID-19 results in inflammation and fluid in the lungs. COVID-19 looks very similar to other viral and bacterial pneumonias on chest radiographs, which makes it difficult to diagnose. This computer vision model for detection and localization of COVID-19 would help doctors provide a quick and confident diagnosis. As a result, patients could get the right treatment before the most severe effects of the virus take hold.


Currently, COVID-19 can be diagnosed via polymerase chain reaction to detect genetic material from the virus or chest radiograph. However, it can take a few hours and sometimes days before the molecular test results are back. By contrast, chest radiographs can be obtained in minutes. While guidelines exist to help radiologists differentiate COVID-19 from other types of infection, their assessments vary. In addition, non-radiologists could be supported with better localization of the disease, such as with a visual bounding box.


In this competition, the task is to identify and localize COVID-19 abnormalities on chest radiographs. In particular, categorization of the radiographs as negative for pneumonia or typical, indeterminate, or atypical for COVID-19.

* train_study_level.csv - the train study-level metadata, with one row for each study, including correct labels.
* train_image_level.csv - the train image-level metadata, with one row for each image, including both correct labels and any bounding boxes in a dictionary format. Some images in both test and train have multiple bounding boxes.
* sample_submission.csv - a sample submission file containing all image- and study-level IDs.
* train folder - comprises 6334 chest scans in DICOM format, stored in paths with the form study/series/image
* test folder - The hidden test dataset is of roughly the same scale as the training dataset. Studies in the test set may contain more than one label.

# Content table

1. Importing the libraries
2. Importing the datasets
3. Data exploration
4. Read Dicom files
5. Feature engineering

# Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the datasets

In [78]:
train_image_level = pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")
train_study_level = pd.read_csv("../input/siim-covid19-detection/train_study_level.csv")

# Data exploration

Let's have a look inside the train_image_level.

In [79]:
train_image_level.head()

,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e


In [80]:
train_image_level.describe()

,id,boxes,label,StudyInstanceUID
count,6334,4294,6334,6334
unique,6334,4294,4295,6054
top,8f6792bb8de4_image,"[{'x': 2219.59459, 'y': 975.56757, 'width': 61...",none 1 0 0 1 1,0fd2db233deb
freq,1,1,2040,9


There are 6334 unique values in the train_image_level dataframe.

In [81]:
train_study_level.head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1


In [82]:
train_study_level.describe()

,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
count,6054.000000,6054.000000,6054.000000,6054.000000
mean,0.276842,0.471589,0.173274,0.078295
std,0.447475,0.499233,0.378515,0.268658
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


There are 6054 rows in the train_study_level dataframe. The number of unique values in study dataframe differs from the unique values in the images dataframe. Let's check how many studies have more than 1 image linked.

In [84]:
train_study_level_key = train_study_level.id.str[:-6]
merged_right = pd.merge(left = train_study_level, right = train_image_level, how = 'right', left_on = train_study_level_key, right_on = 'StudyInstanceUID')

In [72]:
#merged_right = merged_right.drop(['id_y', 'StudyInstanceUID'], axis=1)
#merged_right.id_x = merged_right.id_x.str[:-6]
#merged_right.rename(columns={"id_x": "id"})

In [86]:
merged_right.boxes[0]

"[{'x': 789.28836, 'y': 582.43035, 'width': 1026.65662, 'height': 1917.30292}, {'x': 2245.91208, 'y': 591.20528, 'width': 1094.66162, 'height': 1761.54944}]"

In [111]:
merged_right[merged_right.groupby('StudyInstanceUID')['id_y'].transform('size') > 1].sort_values('StudyInstanceUID')

,id_x,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,id_y,boxes,label,StudyInstanceUID
2862,00f9e183938e_study,0,0,0,1,74077a8e3b7c_image,"[{'x': 2175.24285, 'y': 1123.72368, 'width': 4...",opacity 1 2175.24285 1123.72368 2607.50603 162...,00f9e183938e
2490,00f9e183938e_study,0,0,0,1,6534a837497d_image,NaN,none 1 0 0 1 1,00f9e183938e
2119,0142feaef82f_study,0,0,1,0,55e22c0c5de0_image,"[{'x': 455.99999, 'y': 1480.00008, 'width': 26...",opacity 1 455.99999 1480.00008 722.39998 2437....,0142feaef82f
6061,0142feaef82f_study,0,0,1,0,f5451a98d684_image,NaN,none 1 0 0 1 1,0142feaef82f
3880,0369e0385796_study,0,1,0,0,9e4824fcee2e_image,"[{'x': 817.77961, 'y': 1075.34501, 'width': 64...",opacity 1 817.77961 1075.34501 1467.08961 2075...,0369e0385796
...,...,...,...,...,...,...,...,...,...
1600,fc45007f145a_study,0,1,0,0,4123a71d9796_image,"[{'x': 889.45144, 'y': 282.39441, 'width': 825...",opacity 1 889.45144 282.39441 1714.51125 1585....,fc45007f145a
827,fd92c6f2b2e6_study,1,0,0,0,218bcf950372_image,NaN,none 1 0 0 1 1,fd92c6f2b2e6
5735,fd92c6f2b2e6_study,1,0,0,0,e6cc65d9de1d_image,NaN,none 1 0 0 1 1,fd92c6f2b2e6
3277,ffcb4630f46f_study,0,1,0,0,84ed5f7f71bf_image,"[{'x': 1721.27651, 'y': 974.09667, 'width': 12...",opacity 1 1721.27651 974.09667 2999.21998 2681...,ffcb4630f46f


In [76]:
train_image_level.sort_values('').head()

,id,boxes,label,StudyInstanceUID
2498,65761e66de9f_image,"[{'x': 720.65215, 'y': 636.51048, 'width': 332...",opacity 1 720.65215 636.51048 1052.84563 1284....,00086460a852
2010,51759b5579bc_image,NaN,none 1 0 0 1 1,000c9c05fd14
6084,f6293b1c49e2_image,NaN,none 1 0 0 1 1,00292f8c37bd
1175,3019399c31f4_image,NaN,none 1 0 0 1 1,005057b3f880
4605,bb4b1da810f3_image,"[{'x': 812.54698, 'y': 1376.41291, 'width': 62...",opacity 1 812.54698 1376.41291 1435.14793 1806...,0051d9b12e72


In [77]:
train_study_level.sort_values('id').head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1


# Read Dicom files

# Feature engineering